# ref
[#1 初心者向け講座 データと課題を理解してSubmitする!](https://www.guruguru.science/competitions/22/discussions/7319eed9-c403-4565-8f59-e148ec39c3f9/)

In [1]:
import os
from glob import glob

import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from contextlib import contextmanager
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil

%matplotlib inline


# ref: Kaggleコード遺産 https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5 
class Timer:
    def __init__(self, logger=None, format_str="{:.3f}[s]", prefix=None, suffix=None, sep=" "):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)


def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

# 再現性確保!
seed_everything(427)

In [2]:
from pathlib import Path
# data_dir
DATA_DIR = Path("../data/")

In [3]:
# 学習用のログデータと正解ラベル
train_log_df = pl.read_csv(DATA_DIR / "train_log.csv")
train_label_df = pl.read_csv(DATA_DIR / "train_label.csv")

# 宿のデータ
yado_df = pl.read_csv(DATA_DIR / "yado.csv")

# テスト期間のログデータ
test_log_df = pl.read_csv(DATA_DIR / "test_log.csv")

sample_submission_df = pl.read_csv(DATA_DIR / "sample_submission.csv")

# 画像のデータ
image_df = pl.read_parquet(DATA_DIR / "image_embeddings.parquet")

In [4]:
# すべてのログデータはあとあと参照をするので先に作っておきます.
whole_log_df = pl.concat([train_log_df, test_log_df])

In [5]:
# 宿の出現回数を計算
yad_count = train_log_df.group_by("yad_no").agg(pl.count().alias("yad_count"))


# 宿のマスター情報と紐づけて
_df = yado_df
_df = _df.join(yad_count, on="yad_no", how="left")

# left join した際の null を 0 に置き換え
_df = _df.with_columns(
    pl.when(
        pl.col("yad_count").is_null()
    )
    .then(0)
    .otherwise(pl.col("yad_count"))
    .alias("yad_count")
)

# 出現回数が多い順に並び変え
_df = _df.sort("yad_count", descending=True)

# この状態で県CDごとに上位30件を取得する
ken_top_30 = _df.group_by("ken_cd").head(30)

In [6]:
def create_session_yad_df(input_df: pl.DataFrame):
    session_last = input_df.group_by("session_id").agg([
        pl.col("*").last()
    ]).join(yado_df, on="yad_no", how="left")

    # 県の上位30個の宿を一番最後の宿情報 `session_last` に紐づけ
    out_df = session_last.select(
        ["session_id", "ken_cd"]
    ).join(
        ken_top_30.select(
            ["ken_cd", "yad_no"]
        )
        ,on="ken_cd"
        ,how="left"
    ).select(
        ["session_id", "yad_no"]
    )

    # 使うのはセッションと宿の関係
    out_df = out_df.select(["session_id", "yad_no"])

    # ランダムに付け加えたもの以外・同一ログに出現する宿を候補に入れる
    out_df = pl.concat([out_df, input_df.select(["session_id", "yad_no"])])

    # 重複は意味がないので消す
    out_df = out_df.unique()

    # 見た目をそろえるために session / yad の順番でソート
    out_df = out_df.sort(["session_id", "yad_no"])

    return out_df

In [7]:
with Timer(prefix="train session yado..."):
    train_session_yad_df = create_session_yad_df(input_df=train_log_df)

# 予測の際には session ごとに yado に対しての予約確率を出さなくてはなりませんから、同じように session - yado の組を作ります。
# ただし学習時と同じような組で良いか? は議論が必要かもしれません. (ここに現れない宿は予測対象に絶対入らないため)

with Timer(prefix="test session yado..."):
    test_session_yad_df = create_session_yad_df(input_df=test_log_df)

train session yado... 2.511[s]
test session yado... 1.431[s]


In [8]:
# テストデータへの後処理: ログの一番最後の宿の削除
# 「ログデータの一番最後の宿は必ず正解ラベルではない」という制約条件があるので、セッションの最後の宿は取り去る処理を行っておきましょう。
def remove_last_yad_id(session_yad_df: pl.DataFrame):
    # セッション中の一番最後の宿の組を作成
    last_yad_df = whole_log_df.group_by("session_id").agg([pl.col("*").last()])

    # 最後であることがわかるようにラベル is_last を付与
    last_yad_df = last_yad_df.with_columns(pl.lit(1).alias("is_last"))

    # 引数の session - yad の組み合わせとマージして
    merged = session_yad_df.join(last_yad_df, on=["session_id", "yad_no"], how="left")

    # is_last ではないデータ（is_last is null）のみに絞る
    out_df = merged.filter(pl.col("is_last").is_null()).select(["session_id", "yad_no"])

    return out_df

In [9]:
test_session_yad_df = remove_last_yad_id(test_session_yad_df)

# 特徴量の作成

In [10]:
# ======= 1:セッションが持つ情報 -- セッションの長さ情報 =======
session_length = whole_log_df.group_by("session_id").agg(pl.max("seq_no").alias("session_length"))

def create_session_length_feature(input_df: pl.DataFrame) -> pl.DataFrame:
    session_length = whole_log_df.group_by("session_id").agg(pl.max("seq_no").alias("session_length"))
    out_df = input_df.join(session_length, on="session_id", how="left").select(["session_length"]).drop(["session_id"])
    return out_df

# 2回同じ関数を使って特徴を作ったとき同一のデータができるか? をテストしておくと少し安心です
assert create_session_length_feature(train_session_yad_df).equals(create_session_length_feature(train_session_yad_df), null_equal=True)

# ======= 2:宿の情報 -- 数値系特徴 =======
def create_yado_numaric_feature(input_df: pl.DataFrame) -> pl.DataFrame:
    num_columns = [
        'yad_type',
        'total_room_cnt',
        'wireless_lan_flg',
        'onsen_flg',
        'kd_stn_5min',
        'kd_bch_5min',
        'kd_slp_5min',
        'kd_conv_walk_5min',
    ]

    # `yad_no`をキーとして結合
    out_df = input_df.join(yado_df.select(["yad_no", *num_columns]), on="yad_no", how="left").drop("yad_no")
    return out_df

assert create_yado_numaric_feature(train_session_yad_df).equals(create_yado_numaric_feature(train_session_yad_df))

# ======= 3: 宿の情報 -- wid cd の label encoding =======
# LabelEncoding は scikit-learn に変換ロジックが用意されていますのでそれを利用するのが便利です。
from sklearn.preprocessing import LabelEncoder

# 使い方はシンプルで, LabelEncoder を定義したあと fit_trainsform で与えられた配列を数値 Label に変換します。
le = LabelEncoder()

wid_cd_label = le.fit_transform(yado_df["wid_cd"])

def create_yad_wid_cd_feature(input_df: pl.DataFrame) -> pl.DataFrame:
    source_df = pl.DataFrame({
        "yad_no": yado_df["yad_no"],
        "wid_cd_label": wid_cd_label
    })

    out_df = input_df.join(source_df, on="yad_no", how="left").drop("yad_no")
    return out_df

assert create_yad_wid_cd_feature(train_session_yad_df).equals(create_yad_wid_cd_feature(train_session_yad_df))

In [11]:
# ログデータの中に宿Noは入っているのか

def create_is_in_log_feature(input_df: pl.DataFrame) -> pl.DataFrame:
    # ログデータ全体から重複を排除して`is_in_log`フラグを付与
    _df = whole_log_df.select(["session_id", "yad_no"]).unique()
    _df = _df.with_columns(pl.lit(1).alias("is_in_log"))

    # 入力された session - yad と結合し、欠損値を0で置き換え
    out_df = input_df.join(_df, on=["session_id", "yad_no"], how="left").with_columns(
        pl.when(
            pl.col("is_in_log").is_null()
        )
        .then(0)
        .otherwise(pl.col("is_in_log"))
        .alias("is_in_log")
    )

    return out_df

assert create_is_in_log_feature(train_session_yad_df).equals(create_is_in_log_feature(train_session_yad_df))

In [12]:
# 対象と同じ地域を見ているか

def create_option_yad_and_last_yado_is_same_region_feature(input_df: pl.DataFrame) -> pl.DataFrame:

    # 0: 地域のカラム名を指定(あとでべつの列でもできるように!)
    region_column = "sml_cd"

    # 1: セッション宿に地域を紐づけ
    session_yad_region_df = input_df.join(yado_df.select(["yad_no", region_column]), on="yad_no", how="left")

    # 2: ログデータを使って, セッションの一番最後のレコードに地域を紐づけ
    last_session_yad_df = whole_log_df.group_by("session_id").agg(pl.col("yad_no").last().alias("yad_no"))
    last_session_yad_df = last_session_yad_df.join(yado_df.select(["yad_no", region_column]), on="yad_no", how="left")

    # 3: セッション宿のセッションに, 一番最後の宿の地域を紐づけ
    last_yad_region = session_yad_region_df.select("session_id").join(last_session_yad_df.select(["session_id", region_column]), on="session_id", how="left")

    # 4: 1 と 3 の地域が一致している == 一番最後の宿の地域と候補の宿の地域が一緒!
    idx = session_yad_region_df[region_column] == last_yad_region[region_column]
    
    out_df = pl.DataFrame({"same": idx.cast(int)}).with_columns(pl.col("same").alias(f"{region_column}_is_same"))


    return out_df

assert create_option_yad_and_last_yado_is_same_region_feature(test_session_yad_df)\
    .equals(create_option_yad_and_last_yado_is_same_region_feature(test_session_yad_df))

In [13]:
# 画像の特徴を使う

emb_columns = [col for col in image_df.columns if "emb" in col]

# 次元圧縮

from sklearn.decomposition import TruncatedSVD

# 今回は 32 次元にすることにしました
img_svd = TruncatedSVD(n_components=32)

# 使い方は簡単で, shape = (n_data, n_features,) の numpy 配列を渡せばOKです
z = img_svd.fit_transform(image_df.select(emb_columns).to_numpy())

# 今回は 32 次元を指定したので z は (n_data, 32,) 次元の配列になります
# 512 → 32 次元に圧縮ができました!

# session - yado と紐付けるときは一度代表値による集約をして yado ごとの特徴に変換します (今回は max)
svd_img_df = pl.DataFrame(data=z)

max_svd_df = svd_img_df.group_by(image_df.select(["yad_no"])).max()

# その後 yad_no で left join!
out_df = train_session_yad_df.join(max_svd_df, on="yad_no", how="left").drop("yad_no")


In [14]:
def create_yado_image_feature(input_df: pl.DataFrame) -> pl.DataFrame:
    out_df = input_df.join(max_svd_df, on="yad_no", how="left").drop("yad_no")

    return out_df.rename({col: "yad_img_max_" + col for col in out_df.columns if col != "yad_no"})

assert create_yado_image_feature(train_session_yad_df).equals(create_yado_image_feature(train_session_yad_df))

In [15]:
# 特徴量のマージ

def create_feature(input_df: pl.DataFrame) -> pl.DataFrame:
    functions = [
        create_session_length_feature,
        create_yado_numaric_feature,
        create_yad_wid_cd_feature,
        create_is_in_log_feature,
        create_yado_image_feature,
        create_option_yad_and_last_yado_is_same_region_feature,
    ]

    out_df = pl.DataFrame()
    for func in functions:
        _df = func(input_df)
        if out_df.width == 0:  # 最初の関数の出力をそのまま使う
            out_df = _df
        else:  # 以降はキー列を除いて結合
            out_df = pl.concat([out_df, _df.drop(["session_id", "yad_no"])], how='horizontal')

    return out_df

In [16]:
# 実行して train / test 用の特徴量を作ります.

with Timer(prefix="train..."):
    train_feat_df = create_feature(train_session_yad_df)

with Timer(prefix="test..."):
    test_feat_df = create_feature(test_session_yad_df)

train... 1.610[s]
test... 1.006[s]


In [17]:
# 学習用データへの後処理

# 正解ラベル train_label_df の組み合わせを付与
_df = train_session_yad_df.clone()

# 重複を削除して
_df = _df.unique()

# 正解ラベルに含まれているレコードの index を配列で取得して
target_index = train_label_df.with_columns(pl.lit(1).alias("target"))

# 正解Indexに含まれている場合 1 / そうでないと 0 のラベルを作成
_df = _df.join(target_index, on=["session_id", "yad_no"], how="left")
_df = _df.with_columns([
    pl.when(pl.col("target").is_null()).then(0).otherwise(pl.col("target")).alias("target")
])
# 見た目を揃えるために session / yad でソートしておく
_df = _df.sort(["session_id", "yad_no"])

train_session_yad_df = _df.clone()


In [18]:
# X = train_feat_df.to_numpy()

In [19]:
# session_idが紛れ込むからここで消す

X = train_feat_df.drop("yad_img_max_session_id").to_numpy()

In [20]:
y = train_session_yad_df.select("target").to_numpy().flatten()

In [21]:
# モデルの学習

from sklearn.model_selection import GroupKFold

fold = GroupKFold(n_splits=5)
cv = fold.split(X, y, groups=train_session_yad_df.select("session_id").to_numpy())
cv = list(cv) # split の返り値は generator なので list 化して何度も iterate できるようにしておく

import lightgbm as lgbm
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score, f1_score, mean_absolute_error, mean_squared_error, \
    r2_score, mean_squared_log_error, median_absolute_error, explained_variance_score, cohen_kappa_score, \
    average_precision_score, precision_score, recall_score


def binary_metrics(y_true: np.ndarray,
                   predict_probability: np.ndarray,
                   threshold=.5) -> dict:
    """
    calculate binary task metrics
    Args:
        y_true:
            target. shape = (n_data,)
        predict_probability:
            predict value. be probability prediction for log_loss, roc_auc, etc.
        threshold:
            Thresholds for calculating the metrics that need to be evaluated as labels
    Returns:
        metrics metrics dictionary. the key is metric name, and the value is score.
    """
    predict_label = np.where(predict_probability > threshold, 1, 0)
    none_prob_functions = [
        accuracy_score,
        f1_score,
        precision_score,
        recall_score
    ]

    prob_functions = [
        roc_auc_score,
        log_loss,
        average_precision_score
    ]

    scores = {}
    for f in none_prob_functions:
        score = f(y_true, predict_label)
        scores[str(f.__name__)] = score
    for f in prob_functions:
        score = f(y_true, predict_probability)
        scores[f.__name__] = score

    return scores

def fit_lgbm(X, 
             y, 
             cv, 
             params: dict=None):
    """lightGBM を CrossValidation の枠組みで学習を行なう function"""

    # パラメータがないときは、空の dict で置き換える
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    # training data の target と同じだけのゼロ配列を用意
    oof_pred = np.zeros((n_records, ), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        print(f"-- start fold {i}")
        # この部分が交差検証のところです。データセットを cv instance によって分割します
        # training data を trian/valid に分割
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMClassifier(**params, verbose=0)

        with Timer(prefix="fit fold={} ".format(i)):

            # cv 内で train に定義された x_train で学習する
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    callbacks=[
                        lgbm.early_stopping(stopping_rounds=50, verbose=True),
                        lgbm.log_evaluation(period=50, ),
                    ],)

        # cv 内で validation data とされた x_valid で予測をして oof_pred に保存していく
        # oof_pred は全部学習に使わなかったデータの予測結果になる → モデルの予測性能を見る指標として利用できる
        pred_i = clf.predict_proba(x_valid)[:, 1]
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        score = binary_metrics(y_valid, pred_i)
        print(f" - fold{i + 1} - {score}")

    score = binary_metrics(y, oof_pred)

    print("=" * 50)
    print(f"FINISHI: Whole Score: {score}")
    return oof_pred, models

In [22]:
params = {
    # 目的関数. これの意味で最小となるようなパラメータを探します. 
    "objective": "binary", 

    # 木の最大数
    "n_estimators": 10000, 

     # 学習率. 小さいほどなめらかな決定境界が作られて性能向上に繋がる場合が多いです、
    "learning_rate": .1,

    # 特徴重要度計算のロジック(後述)
    "importance_type": "gain", 
    "random_state": 427,
}

oof, models = fit_lgbm(X, y=y, params=params, cv=cv)

-- start fold 0
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.0496838
[100]	valid_0's binary_logloss: 0.0494446
[150]	valid_0's binary_logloss: 0.0493173
[200]	valid_0's binary_logloss: 0.049247
[250]	valid_0's binary_logloss: 0.0492173
[300]	valid_0's binary_logloss: 0.0491926
[350]	valid_0's binary_logloss: 0.0491839
[400]	valid_0's binary_logloss: 0.0491776
[450]	valid_0's binary_logloss: 0.0491813
Early stopping, best iteration is:
[414]	valid_0's binary_logloss: 0.0491734
fit fold=0  80.106[s]
 - fold1 - {'accuracy_score': 0.9843115214683115, 'f1_score': 0.18545052128836859, 'precision_score': 0.6578512396694215, 'recall_score': 0.10793952132347956, 'roc_auc_score': 0.9322439638032329, 'log_loss': 0.04917341217231733, 'average_precision_score': 0.3904399586399856}
-- start fold 1
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.049778
[100]	valid_0's binary_logloss: 0.049521
[150]	valid_0'

In [42]:
# 結果を解釈する

# 手元でスコアを見積る

def create_top_10_yad_predict(predict, session_yad_df):
    predict_series = pl.Series("predict", predict)
    predict_df = predict_series.to_frame()
    _df = pl.concat([session_yad_df.select(["session_id", "yad_no"]), predict_df], how="horizontal")

    # セッションごとに予測確率の高い順に yad_no の配列を作成
    _agg = _df.sort("predict", descending=True).group_by("session_id").agg(pl.col("yad_no").map_elements)

    # データフレームの形式を整える
    out_df = _agg.map_rows(lambda x: x[:10])

    return out_df

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k for a single actual value.

    Parameters:
    actual : int
        The actual value that is to be predicted
    predicted : list
        A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns:
    float
        The average precision at k
    """
    if actual in predicted[:k]:
        return 1.0 / (predicted[:k].index(actual) + 1)
    return 0.0

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k for lists of actual values and predicted values.

    Parameters:
    actual : list
        A list of actual values that are to be predicted
    predicted : list
        A list of lists of predicted elements (order does matter in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns:
    float
        The mean average precision at k
    """
    return sum(apk(a, p, k) for a, p in zip(actual, predicted)) / len(actual)

In [43]:
oof_label_df = create_top_10_yad_predict(predict=oof, session_yad_df=train_session_yad_df)

# いま作成した session_id と同じ並びで train_label を並び替え
train_label = train_label_df.set_index("session_id").loc[oof_label_df.index]["yad_no"].values

# MAPK (k=10) として計算
oof_score = mapk(actual=train_label, predicted=oof_label_df.values.tolist(), k=10)

print(f"OOF Score: {oof_score:.4f}")

/tmp/ipykernel_162367/3189812188.py:11: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  _agg = _df.sort("predict", descending=True).group_by("session_id").agg(pl.col("yad_no").apply(list))
/tmp/ipykernel_162367/3189812188.py:14: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_rows`.
  out_df = _agg.apply(lambda x: x[:10])


AttributeError: 'DataFrame' object has no attribute 'set_index'